In [55]:
import numpy as np
import matplotlib as plt
# import huggingface_hub as hub
# from huggingface_hub import hf_hub_download
%pip install datasets
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import HashingVectorizer

import gensim
from sklearn.decomposition import PCA

from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
 
warnings.filterwarnings(action='ignore')

import nltk
from nltk.corpus import stopwords

import string
 
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('stopwords')


Note: you may need to restart the kernel to use updated packages.


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\jarod\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jarod\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jarod\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
ds = load_dataset("artem9k/ai-text-detection-pile")

In [3]:
type(ds)

datasets.dataset_dict.DatasetDict

In [ ]:
# Load the data into targets and texts then split into train and test sets

unsplit = ds['train']
# print(unsplit[83:86])

targets = unsplit['source']
ids = unsplit['id']
texts = unsplit['text']

# print(unsplit[1])

X_train_text, X_test_text, t_train, t_val  = train_test_split(texts, targets, test_size = 0.2, random_state=2013)


In [62]:
# Get rid of punctuation and other unneeded words
stop_words = set(stopwords.words('english'))
punctuation = set(string.punctuation)
exclude_words = set(['``', "''", "'s", "n't", '’', '“', '”'])

def preprocess_text(text):
    # Tokenize the text into words (assuming 'text' is a single document)
    tokens = word_tokenize(text.lower())  # Convert to lowercase for consistency
    
    # Filter out stopwords, punctuation, and custom exclude words
    words = [word for word in tokens if word not in stop_words and word not in punctuation and word not in exclude_words]
    
    return words


# Method of using a Hashing Vectorizer:

In [ ]:
# Feature Extraction methods
# print(X_train[20])
# print(t_train[20])

# Try a HashingVectorizer
vectorizer = HashingVectorizer(decode_error='ignore', n_features=2 ** 18, lowercase=True)
X_train = vectorizer.transform(X_train_text)

# Method of Word Embedding the Tokenized Text:

In [70]:
# Try a word embedding with self attention
foo = X_train_text[slice(0, 5000, None)]

# tokenized_text = [word_tokenize(doc.lower()) for doc in foo]  # Convert to lowercase for consistency

# Now `tokenized_text` is a list of lists of words

tokenized_text = [preprocess_text(doc) for doc in foo]

model1 = gensim.models.Word2Vec(tokenized_text, vector_size=100, window=5, min_count=1, workers=4)


model1.save('foo.embedding')
new_model = gensim.models.Word2Vec.load('foo.embedding')


# Check the vocabulary keys (words)
myVocab = list(new_model.wv.key_to_index.keys())
# print(words[:50])  # Display first 50 words




In [71]:
# wordkeys = list(new_model.wv.key_to_index.keys())

# Print the first 10 words as an example
# print(len(wordkeys))
print(new_model.wv['car'])
print(len(new_model.wv['car']))

[-0.10184555  0.74403304  0.42221144  0.05052288  0.16777846 -1.1204435
 -0.0352641   1.5608088  -1.5689924  -1.0173714  -0.05143084 -1.1016809
  0.6849086   0.6459909   0.07042554 -1.473673   -0.20201273 -0.6803107
 -0.44041193 -0.7329893   0.8990882  -0.68871355  0.92601204 -1.1919641
 -0.10772374  0.5810186  -0.47519147 -0.99869955 -0.43956515  0.36634326
  0.5862961   0.26830578 -0.45360813 -0.1832539   0.05729854  0.07933962
 -0.4126892   0.22998612 -0.47502235 -1.9402974   0.47157538  0.4090291
  0.4937879   0.30064827  0.33350912 -0.89447474  0.23144081 -0.79632497
  0.06929835  0.20943744  0.20179234 -0.8212298  -0.26296705 -0.5856572
 -0.11008263 -0.11561868  0.2960215   0.36112353 -0.7183832   0.47270143
  0.05564627  0.22125627  0.19361156  0.5928681  -0.54815227  1.6625143
  0.31272408  0.17887233 -0.7854722   0.4225436  -0.83799046  0.2028884
  1.0658158  -0.01606462  0.5385432   0.38711792 -0.361795    0.5317837
 -0.18167926  0.40007263  0.1267494   0.8341262  -1.0056862 

# Make a self attention matrix:


TypeError: expected string or bytes-like object, got 'list'